In [131]:
import pandas as pd
import datetime
import requests
from bs4 import BeautifulSoup
import xlrd as xl
import urllib.request
import json
import numpy
import os

In [132]:
df_AENA_pasajeros = pd.read_pickle("./raw_df_AENA_pasajeros.pkl")
df_AENA_operaciones = pd.read_pickle("./raw_df_AENA_operaciones.pkl")
df_AENA_mercancias = pd.read_pickle("./raw_df_AENA_mercancias.pkl")

In [133]:
url_AENA_IATA_aeropuertos = "https://raw.githubusercontent.com/abediaga/AENA/main/AENA_IATA_aeropuertos.json"
file_name = "AENA_IATA_aeropuertos.json"

urllib.request.urlretrieve(url_AENA_IATA_aeropuertos, file_name)

aeropuertos_ = {}

with open('AENA_IATA_aeropuertos.json') as f:
    aeropuertos = json.load(f)

for aero_key, aero_value in aeropuertos.items():
    #aeropuertos_[aero_key] = aero_value["Nombre"]
    aeropuertos_[aero_value["Nombre"]] = aero_key
    if "Otros" in aero_value.keys():
        #print(aero_value["Otros"])
        for item in aero_value["Otros"]:
            #print(item)
            #aeropuertos_[aero_key] = str(item)
            aeropuertos_[str(item)] = aero_key

In [134]:
#display(df_AENA_pasajeros)

In [135]:
#df_AENA_pasajeros = df_AENA_pasajeros.drop((2021,"Febrero"))
#df_AENA_pasajeros = df_AENA_pasajeros.drop((2021,"Marzo"))
#df_AENA_pasajeros = df_AENA_pasajeros.drop(2021)
#display(df_AENA_pasajeros)

In [136]:
fecha_datos = df_AENA_pasajeros.index.to_list()

ultimo_ano = fecha_datos[-1][0]
ultimo_mes = fecha_datos[-1][1]

now = datetime.datetime.now()
ano_actual = now.year
mes_actual = now.month

if ultimo_mes == "Diciembre":
    ano_ = ultimo_ano + 1
else :
    ano_ = ultimo_ano

In [137]:
has_changed = False

main_url = "https://wwwssl.aena.es/"
url = "https://wwwssl.aena.es/csee/Satellite?SiteName=Estadisticas&anyo=" + str(ano_) + "&c=Page&cid=1144247795704&pagename=Estadisticas%2FEstadisticas&periodoInforme=Mensual"
response = requests.get(url)

soup = BeautifulSoup(response.text, 'html.parser')
contenido = soup.find_all("td", {"class": "link2"})

if len(contenido) > 0:
    for elemento in contenido:
        el = elemento.findAll("a", {"class": "link2"})
        if ".xls" in el[0]['href']:
            mes_ano = el[0].text.strip().partition(" ")
            mes = mes_ano[0]
            ano = int(mes_ano[-1])
            if (ano,mes) not in fecha_datos:
                #obtener datos y agregarlos al dataframe
                link = main_url + el[0]['href']
                file = requests.get(link)
                filename = 'aena_temp.xls'
                open(filename, 'wb').write(file.content)
                Workbook = xl.open_workbook(filename)
                sheets = Workbook.sheet_names()
                hoja_seleccionada = ""
                for sheet in sheets:
                    if ("TRÁFICO" in sheet) or ("TRAFICO" in sheet):
                        hoja_seleccionada = sheet
                if (ano < 2009):
                    df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[0,1,2,3,4,5], usecols=[1,2,5,6,9,10])
                elif (ano == 2009):
                    df = pd.read_excel(filename, sheet_name=hoja_seleccionada, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])
                elif (ano > 2010) and (ano < 2021):
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])
                elif (ano == 2021) and (mes == "Enero"):
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])
                elif (ano == 2021) and (mes == "Febrero"):
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,4,7,8,11,12])
                else:
                    df = pd.read_excel(filename, skiprows=[1,2,3,4,5,6,7,8], usecols=[2,3,7,8,12,13])
                
                df_pasajeros = df[df.columns[0:2]].copy()
                df_pasajeros = df_pasajeros.set_index(df_pasajeros.columns[0])

                df_operaciones = df[df.columns[2:4]].copy()
                df_operaciones = df_operaciones.set_index(df_operaciones.columns[0])

                df_mercancias = df[df.columns[4:6]].copy()
                df_mercancias = df_mercancias.set_index(df_mercancias.columns[0])

                df_pasajeros.index.name = 'Aeropuerto'
                df_operaciones.index.name = 'Aeropuerto'
                df_mercancias.index.name = 'Aeropuerto'

                df_pasajeros = df_pasajeros[df_pasajeros.index.notnull()]
                df_operaciones = df_operaciones[df_operaciones.index.notnull()]
                df_mercancias = df_mercancias[df_mercancias.index.notnull()]
                
                
                for index, row in df_pasajeros.iterrows():
                    index = index.strip()
                    if (index in aeropuertos_.keys()):
                        if type(row.iloc[0]) == numpy.float64:
                            valor = row.iloc[0].astype(int)
                        if type(row.iloc[0]) == float:
                            valor = int(row.iloc[0].astype(int))
                        if type(row.iloc[0]) == str:
                            valor = 0
                        
                        df_AENA_pasajeros.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
                        has_changed = True
                
                for index, row in df_operaciones.iterrows():
                    index = index.strip()
                    if (index in aeropuertos_.keys()):
                        if type(row.iloc[0]) == numpy.float64:
                            valor = row.iloc[0].astype(int)
                        if type(row.iloc[0]) == float:
                            valor = int(row.iloc[0].astype(int))
                        if type(row.iloc[0]) == str:
                            valor = 0
                        
                        df_AENA_operaciones.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
                        has_changed = True
                
                for index, row in df_mercancias.iterrows():
                    index = index.strip()
                    if (index in aeropuertos_.keys()):
                        if type(row.iloc[0]) == numpy.float64:
                            valor = row.iloc[0].astype(int)
                        if type(row.iloc[0]) == float:
                            valor = int(row.iloc[0].astype(int))
                        if type(row.iloc[0]) == str:
                            valor = 0
                        
                        df_AENA_mercancias.loc[(ano,mes),aeropuertos_[index]] = row.iloc[0]
                        has_changed = True
                

In [138]:
#display(df_AENA_pasajeros)

MAD        BCN       LPA       TFS        PMI  \
Año  Mes                                                              
2004 Enero      2665773.0  1502099.0  828458.0  784481.0   687430.0   
     Febrero    2839210.0  1697096.0  838184.0  776373.0   890473.0   
     Marzo      3107523.0  1958778.0  869869.0  788108.0  1147727.0   
     Abril      3155361.0  2047100.0  792746.0  793833.0  1429425.0   
     Mayo       3211131.0  2144105.0  675820.0  521634.0  2039736.0   
...                   ...        ...       ...       ...        ...   
2020 Noviembre   697692.0   341502.0  305070.0  151486.0   198580.0   
     Diciembre   988144.0   541332.0  348991.0  180787.0   243585.0   
2021 Enero       845168.0   411486.0  234986.0   97926.0   165533.0   
     Febrero     752969.0   286650.0  183157.0   81182.0   141657.0   
     Marzo       971453.0   423774.0  275655.0  103848.0   272209.0   

                      AGP       ALC       ACE       FUE       TFN  ...  \
Año  Mes                                                           ...   
2004 Enero       629797.0  476511.0  430950.0  313009.0  248771.0  ...   
     Febrero     741498.0  534494.0  427889.0  305360.0  258365.0  ...   
     Marzo       858004.0  647784.0  444459.0  354216.0  261917.0  ...   
     Abril      1057593.0  715807.0  473803.0  328743.0  283991.0  ...   
     Mayo       1162539.0  776707.0  387849.0  282377.0  272835.0  ...   
...                   ...       ...       ...       ...       ...  ...   
2020 Noviembre   157089.0  112618.0  144876.0  121752.0  189058.0  ...   
     Diciembre   208235.0  126143.0  163667.0  137624.0  191181.0  ...   
2021 Enero       168111.0   77325.0   84735.0   90111.0  137863.0  ...   
     Febrero     105172.0   37703.0   54278.0   73957.0  147471.0  ...   
     Marzo       166876.0   61938.0   84980.0  102232.0  203298.0  ...   

                   ODB     JCU     ABC   LECU    QSA   LESB    HSK     RGS  \
Año  Mes                                                                     
2004 Enero       798.0   763.0   215.0    0.0    0.0    NaN    NaN     NaN   
     Febrero    1305.0  1229.0   547.0    0.0    0.0    NaN    NaN     NaN   
     Marzo      3126.0  1350.0   287.0    0.0    0.0    NaN    NaN     NaN   
     Abril      2035.0  1174.0  1646.0    0.0    0.0    NaN    NaN     NaN   
     Mayo       1572.0  1307.0  1604.0    6.0    0.0    NaN    NaN     NaN   
...                ...     ...     ...    ...    ...    ...    ...     ...   
2020 Noviembre   674.0  2804.0   318.0  118.0  265.0  336.0  114.0  2151.0   
     Diciembre   348.0  3464.0   350.0   78.0  369.0  229.0  234.0   383.0   
2021 Enero       170.0  2758.0   183.0   48.0  170.0  206.0  162.0   473.0   
     Febrero     191.0  3600.0   294.0   55.0  229.0  363.0  282.0   360.0   
     Marzo       203.0  4842.0   237.0  250.0  361.0  417.0   45.0   471.0   

                   AEI     RMU  
Año  Mes                        
2004 Enero         NaN     NaN  
     Febrero       NaN     NaN  
     Marzo         NaN     NaN  
     Abril         NaN     NaN  
     Mayo          NaN     NaN  
...                ...     ...  
2020 Noviembre  1431.0  1966.0  
     Diciembre  1759.0  2313.0  
2021 Enero      1407.0  1658.0  
     Febrero    2087.0   802.0  
     Marzo      2793.0  1438.0  

[207 rows x 50 columns]

In [139]:
if has_changed:
    pd.to_pickle(df_AENA_pasajeros, "./raw_df_AENA_pasajeros.pkl")
    pd.to_pickle(df_AENA_operaciones, "./raw_df_AENA_operaciones.pkl")
    pd.to_pickle(df_AENA_mercancias, "./raw_df_AENA_mercancias.pkl")
    os.remove("aena_temp.xls")